# 2022-1 빅데이터와 인공지능 12주차 Reports

## 12주차 1차시

### Question 1
1. K-NN 알고리즘의 하이퍼파라미터(k, 거리 종류, 가중치)를 변경하면서 예측성능(정분류율)을 확인하시오.
2. 시도한 하이퍼파라미터 중 가장 우수한 성능을 보인 하이퍼파라미터를 적용한 Code를 업로드하시오.

In [15]:
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix

In [2]:
# Load datasets
iris_data = load_iris()

In [3]:
# Create dataframes
df_X = pd.DataFrame(data=iris_data.data, columns=iris_data.feature_names)
df_y = pd.DataFrame(data=iris_data.target, columns=['Species'])
df_iris = pd.concat([df_X, df_y], axis=1)
df_iris.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),Species
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [5]:
iris_scaler = MinMaxScaler().fit(df_X)
iris_new = pd.DataFrame(iris_scaler.transform(df_X), columns=iris_data.feature_names)
iris_new.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,0.222222,0.625000,0.067797,0.041667
1,0.166667,0.416667,0.067797,0.041667
2,0.111111,0.500000,0.050847,0.041667
3,0.083333,0.458333,0.084746,0.041667
4,0.194444,0.666667,0.067797,0.041667


In [12]:
# Split train and test datasets
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=0)
for Train_Index, Test_Index in split.split(df_iris, df_iris['Species']):
    iris_train = df_iris.loc[Train_Index]
    iris_test = df_iris.loc[Test_Index]
    X_train, y_train = iris_train.iloc[:, :4], iris_train['Species']
    X_test, y_test = iris_train.iloc[:, :4], iris_test['Species']

In [18]:
# Set hyperparameters by each case
iris_minmax_scaler = MinMaxScaler()
list_k = [i for i in range(1, 6)]
list_weights = ['uniform', 'distance']
list_metric = ['minkowski', 'manhattan', 'euclidean', 'mahalanobis']

In [24]:
for i in list_k:
    for j in list_weights:
        for k in list_metric:
            knn_clf = KNeighborsClassifier(n_neighbors=i, weights=j, metric=k)
            Full_Pipeline_KNN = Pipeline(steps=[('scaler', iris_minmax_scaler), ('Classifier', knn_clf)])
            Full_Pipeline_KNN.fit(X_train, y_train)
            iris_test_pred = Full_Pipeline_KNN.predict(X_test)
            print("Accuracy = ", Full_Pipeline_KNN.score(X_test, y_test))
            """
            X_test and y_test don't have save shape --> X_test.shape: (120, 4), y_test.shape: (30,)
            """
            

### Question 2
* Boston 집값 예측을 위한 k-NN 알고리즘을 프로그래밍하고, Code를 업로드하세요.

## 12주차 2차시
### Question 3
* 학습자료로 제공된 "Multiple_regression.csv" 데이터에 대한 선형회귀모형 Code를 업로드하세요.

## 12주차 3차시
### Question 4
1. 의사결정나무의 하이퍼파라미터들을 변경하면서 분류 모형과 예측모형의 성능 변화를 확인하시오.
2. 실습한 Code를 업로드하세요